In [2]:
import pandas as pd
d=pd.read_csv('Historical_DOB_Permit_Issuance.csv')
print(d.head())

C:\Users\jeban\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8,9,10,15,31,33,36,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


  BOROUGH      BIN Number            Street      Job #  Job doc. # Job Type  \
0   BRONX  2118801   2960    WEBSTER AVENUE  201088492           4       NB   
1   BRONX  2096812    100     DEKRUIF PLACE  200716298           2       A2   
2   BRONX  2008604   1898   HARRISON AVENUE  200974650           2       A2   
3   BRONX  2007652   1998     MORRIS AVENUE  200278118           2       A1   
4   BRONX  2084155    565  WEST  235 STREET  201119173           2       A2   

  Self_Cert Block  Lot  ... Owner’s House State  Owner’s House Zip Code  \
0       NaN  3274    4  ...                 NaN                     NaN   
1       NaN  5141  120  ...                 NaN                     NaN   
2       NaN  2869   87  ...                 NaN                     NaN   
3       NaN  2807   15  ...                 NaN                     NaN   
4         Y  5794  484  ...                 NaN                     NaN   

   Owner's Phone #           DOBRunDate   Latitude  Longitude  \
0        

In [3]:
d.count()

BOROUGH                             2428526
BIN                                 2428526
Number                              2428522
Street                              2428522
Job #                               2428526
Job doc. #                          2428526
Job Type                            2428526
Self_Cert                            900685
Block                               2428028
Lot                                 2428019
Community Board                     2425674
Postcode                            2427476
Bldg Type                           2374247
Residential                          772772
Special District 1                   186592
Special District 2                     2639
Work Type                           1974557
Permit Status                       2419621
Filing Status                       2428526
Permit Type                         2428525
Permit Sequence #                   2428526
Permit Subtype                      1418233
Oil Gas                         

# Question 2.1

In [6]:
g=d.groupby('Filing Status').count()[['BIN']]
m= (g['BIN']/g['BIN'].sum())*100
m

Filing Status
INITIAL    76.243079
RENEWAL    23.756921
Name: BIN, dtype: float64

# Question 2.2

In [7]:
from datetime import datetime
import numpy as np
df=pd.DataFrame(d, columns=['BOROUGH','Expiration Date','Issuance Date'])
df['Expiration Date']= pd.to_datetime(df['Expiration Date' ])
df['Expiration Date'].head()
df['Issuance Date']=pd.to_datetime(df['Issuance Date' ])
df['Days']=df['Expiration Date']-df['Issuance Date']
df['Days'].head()
df['Yes'] = np.where(df['Days']>= '365', True, False)
#df.head()
y=df.groupby('BOROUGH').count()[['Yes']]
n= (y['Yes']/y['Yes'].sum())*100
n

BOROUGH
BRONX             8.854548
BROOKLYN         21.922104
MANHATTAN        41.506823
QUEENS           21.329234
STATEN ISLAND     6.387290
Name: Yes, dtype: float64

# Question 2.3

In [9]:
df1=pd.DataFrame(d, columns=['Issuance Date','Permit Status'])
df1['Issuance Date']=pd.to_datetime(df1['Issuance Date' ])
df2=df1[(df1['Issuance Date'] > '1990-01-01') & (df1['Issuance Date'] < '2012-12-31')]
#lm=df2.groupby('Permit Status').count()[['Issuance Date']]
df2['Year'] = pd.DatetimeIndex(df2['Issuance Date']).year
df2['True'] = np.where(df2['Permit Status']== 'ISSUED', True, False)
dff = df2[df2['True'] == True]
dff['Year'] = pd.DatetimeIndex(dff['Issuance Date']).year
x1 = dff.groupby(['Year'],as_index=False).agg({'Permit Status': 'count'})
import statsmodels.api as sm
xx= x1.iloc[:,0]
yy= x1.iloc[:,1]
model = sm.OLS(xx,yy)
results = model.fit()
print(results.summary())


<ipython-input-9-ef364796c9ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Year'] = pd.DatetimeIndex(df2['Issuance Date']).year
<ipython-input-9-ef364796c9ab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['True'] = np.where(df2['Permit Status']== 'ISSUED', True, False)
<ipython-input-9-ef364796c9ab>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

                                 OLS Regression Results                                
Dep. Variable:                   Year   R-squared (uncentered):                   0.866
Model:                            OLS   Adj. R-squared (uncentered):              0.860
Method:                 Least Squares   F-statistic:                              142.0
Date:                Fri, 30 Oct 2020   Prob (F-statistic):                    4.54e-11
Time:                        15:53:02   Log-Likelihood:                         -184.36
No. Observations:                  23   AIC:                                      370.7
Df Residuals:                      22   BIC:                                      371.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

# Question 2.4 

In [12]:
a1=pd.DataFrame(d, columns=['Issuance Date','Permit Status','Expiration Date', 'Job Start Date'])
a1['Issuance Date']=pd.to_datetime(a1['Issuance Date'])
a1['Expiration Date']=pd.to_datetime(a1['Expiration Date'])
a1['Job Start Date'] = pd.to_datetime(a1['Job Start Date'], errors='coerce')
a1= a1.dropna(subset=['Job Start Date'])
a1['Days']=a1['Expiration Date']-a1['Issuance Date']
a1['Month'] = pd.DatetimeIndex(a1['Job Start Date']).month
a1['Mmonth']= np.where(a1['Month']== 11, "November", "Others")
a1['Issued'] = np.where(a1['Permit Status']== 'ISSUED', "Issued", "Not Issued")
a1['More60'] = np.where(a1['Days']<= "60 Days", "Less", "More")
xz =a1.groupby(['Mmonth','More60'],as_index=False).agg({'Permit Status': 'count'})
zz = xz.pivot_table('Permit Status', ['Mmonth'], 'More60')
#Average for 11 months
zz.iloc[1,:]=zz.iloc[1,:]/11
from scipy.stats import chisquare
res= chisquare(zz)
res

Power_divergenceResult(statistic=array([1629.02677778,  711.53778828]), pvalue=array([0.00000000e+000, 9.26329058e-157]))

In [13]:
zz

More60,Less,More
Mmonth,,
November,21829.0,172146.000000
Others,14171.0,188157.545455


# Question 2.4

In [14]:
a3=pd.DataFrame(d, columns=['Filing Status','Owner\'s Business Type'])
a3['Renewed'] = np.where(a3['Filing Status']== 'RENEWAL', "Renewed", "Others")
a4= a3.dropna(subset=['Owner\'s Business Type'])
indexNames = a4[~(a4['Renewed'] == 'Renewed')].index 
# Delete these row indexes from dataFrame
a4.drop(indexNames , inplace=True)
zzz=a4.groupby(['Owner\'s Business Type'],as_index=False).agg({'Renewed': 'count'})
Ratio= (zzz.iloc[1,1]/zzz.iloc[6,1])
Ratio

C:\Users\jeban\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1.0805818771914573

# Question 2.5

In [28]:
a6= pd.DataFrame(d, columns=['Issuance Date','Permit Status','Postcode'])
a6['Year'] = pd.DatetimeIndex(a6['Issuance Date']).year
indexNames = a6[~(a6['Permit Status'] == 'ISSUED')].index 
# Delete these row indexes from dataFrame
a6.drop(indexNames , inplace=True)
indexNames = a6[~(a6['Year'] == 2010)].index 
# Delete these row indexes from dataFrame
a6.drop(indexNames , inplace=True)
a7=a6.groupby(['Postcode'],as_index=False).agg({'Permit Status': 'count'})
a8=pd.read_csv("2010+Census+Population+By+Zipcode+(ZCTA).csv")
a8=a8.rename(columns={"Zip Code ZCTA": "Postcode"}, errors="raise")
inner_join = pd.merge(a7,  
                      a8,  
                      on ='Postcode',  
                      how ='inner') 
a9=inner_join

indexNames = a9[(a9['2010 Census Population'] <= 1000)].index 
# Delete these row indexes from dataFrame
a9.drop(indexNames , inplace=True)
a9['Ratio']=a9['Permit Status']/ a9['2010 Census Population']
a9['Std']= 2* np.std(a9['Ratio'])
a9['Mean']=np.mean(a9['Ratio'])
a9['Diff']= a9['Ratio']- a9['Mean']
a9['True']=a9['Diff'] >= a9['Std']

In [29]:
a9.groupby(['True'],as_index=False).count()


,True,Postcode,Permit Status,2010 Census Population,Ratio,Std,Mean,Diff
0,False,172,172,172,172,172,172,172
1,True,9,9,9,9,9,9,9


# Question 2.6

In [30]:
b1=pd.DataFrame(d, columns=['Issuance Date','Job Start Date'])

b1['Job Start Date'] = pd.to_datetime(b1['Job Start Date'], errors='coerce')
b1['Year'] = pd.DatetimeIndex(b1['Job Start Date']).year
b1['Month'] = pd.DatetimeIndex(b1['Job Start Date']).month
b2= pd.read_excel('season.xlsx')
inner_join = pd.merge(b1,  
                      b2,  
                      on ='Month',  
                      how ='inner') 
b3=inner_join

b3=b3[(b3['Job Start Date'] > '1990-01-01') & (b3['Job Start Date'] < '2012-12-31')]
indexNames = b3[(b3['Season'] == 'Others')].index 
# Delete these row indexes from dataFrame
b3.drop(indexNames , inplace=True)

b4=b3.groupby(['Year', 'Season'],as_index=False).agg({'Job Start Date': 'count'})
b5= b4.set_index(['Year', 'Season']).unstack()
b5['Ratio']= (b5.iloc[:,0] /b5.iloc[:,1]).astype(float)
max(b5['Ratio'])


C:\Users\jeban\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


3.8049555273189326